# Experimental Data Preprocessing -> Considering image_level_labels

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random
import seaborn as sns
import os

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
data_labels = pd.read_csv("/kaggle/input/rsna-2023-abdominal-trauma-detection/image_level_labels.csv")
data_labels.head()

,patient_id,series_id,instance_number,injury_name
0,10004,21057,362,Active_Extravasation
1,10004,21057,363,Active_Extravasation
2,10004,21057,364,Active_Extravasation
3,10004,21057,365,Active_Extravasation
4,10004,21057,366,Active_Extravasation


In [3]:
data_labels.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12029 entries, 0 to 12028
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   patient_id       12029 non-null  int64 
 1   series_id        12029 non-null  int64 
 2   instance_number  12029 non-null  int64 
 3   injury_name      12029 non-null  object
dtypes: int64(3), object(1)
memory usage: 376.0+ KB


In [4]:
data_labels.loc[data_labels["injury_name"] == "Active_Extravasation"]

,patient_id,series_id,instance_number,injury_name
0,10004,21057,362,Active_Extravasation
1,10004,21057,363,Active_Extravasation
2,10004,21057,364,Active_Extravasation
3,10004,21057,365,Active_Extravasation
4,10004,21057,366,Active_Extravasation
...,...,...,...,...
12024,9632,3750,155,Active_Extravasation
12025,9632,3750,156,Active_Extravasation
12026,9632,3750,157,Active_Extravasation
12027,9632,3750,158,Active_Extravasation


In [5]:
data_labels.loc[data_labels["injury_name"] == "Bowel"]

,patient_id,series_id,instance_number,injury_name
105,10065,37324,48,Bowel
106,10065,37324,49,Bowel
107,10065,37324,50,Bowel
108,10065,37324,51,Bowel
109,10065,37324,52,Bowel
...,...,...,...,...
11987,8684,38440,147,Bowel
11988,8684,38440,148,Bowel
11989,8684,38440,149,Bowel
11990,8684,38440,150,Bowel


In [4]:
patients = data_labels["patient_id"].unique().tolist()
len(patients)

246

In [5]:
series = data_labels["series_id"].unique().tolist()
len(series)

330

### We are going to focus only on extravasation injury, considering extravasation injury only and also patients that have both, extravasation and bowel injury, total of 200 patients.

In [6]:
Extravasation = data_labels.loc[data_labels["injury_name"] == "Active_Extravasation"]
patients_extravasation = Extravasation["patient_id"].unique().tolist()
len(patients_extravasation)

200

In [7]:
Extravasation = Extravasation.reset_index(drop=True)
Extravasation

,patient_id,series_id,instance_number,injury_name
0,10004,21057,362,Active_Extravasation
1,10004,21057,363,Active_Extravasation
2,10004,21057,364,Active_Extravasation
3,10004,21057,365,Active_Extravasation
4,10004,21057,366,Active_Extravasation
...,...,...,...,...
6365,9632,3750,155,Active_Extravasation
6366,9632,3750,156,Active_Extravasation
6367,9632,3750,157,Active_Extravasation
6368,9632,3750,158,Active_Extravasation


In [8]:
unique_pairs = Extravasation[['patient_id', 'series_id']].drop_duplicates()
unique_pairs = unique_pairs.reset_index(drop=True)
unique_pairs

,patient_id,series_id
0,10004,21057
1,10004,51033
2,10217,16066
3,10292,14945
4,10494,65369
...,...,...
254,820,11921
255,820,38809
256,8263,30011
257,9528,1989


In [9]:
unique_pairs["patient_id"]

0      10004
1      10004
2      10217
3      10292
4      10494
       ...  
254      820
255      820
256     8263
257     9528
258     9632
Name: patient_id, Length: 259, dtype: int64

In [10]:
def extract_injury_instances(data, patient_id, series_id):
    patient_data = data.loc[(data["patient_id"] == patient_id) & (data["series_id"] == series_id)]  
    return patient_data["instance_number"].tolist()

def image_label_data_cleaned(data, unique_pairs_ps):

    instances = []
    category = [1 for i in range(len(unique_pairs_ps))]
    
    for i, patient in enumerate(unique_pairs_ps["patient_id"]):               
        patient_instances = extract_injury_instances(data, patient, unique_pairs_ps["series_id"][i])
        instances.append(patient_instances)
    
    final_data = pd.DataFrame(list(zip(unique_pairs_ps["patient_id"], unique_pairs_ps["series_id"], instances, category)),
               columns =["patient_id","series_id", "instances", "category"])
    
    return final_data

In [11]:
cleaned_data = image_label_data_cleaned(Extravasation, unique_pairs)

In [12]:
cleaned_data

,patient_id,series_id,instances,category
0,10004,21057,"[362, 363, 364, 365, 366, 367, 368, 369, 370, ...",1
1,10004,51033,"[376, 377, 378, 379, 380, 381, 382, 383, 384, ...",1
2,10217,16066,"[256, 257, 258, 259, 260, 261, 262, 263, 264, ...",1
3,10292,14945,"[20, 21, 22, 23, 24, 25, 26]",1
4,10494,65369,"[292, 293, 294, 295, 296, 297, 298, 299, 300, ...",1
...,...,...,...,...
254,820,11921,"[86, 87, 88, 89, 90]",1
255,820,38809,"[205, 206, 207, 208]",1
256,8263,30011,"[120, 121, 123, 124]",1
257,9528,1989,"[207, 208, 209, 210, 211, 212, 213, 214, 215, ...",1


## Preprocessing pipeline

In [13]:
len(cleaned_data["instances"][256])

4

In [14]:
p_id = str(cleaned_data["patient_id"][0]) + "/" + str(cleaned_data["series_id"][0])

In [16]:
PATH = "/kaggle/input/rsna-2023-abdominal-trauma-detection/train_images/"
total_paths = []
for x in range(len(cleaned_data)):
    paths = []
    for i, instance in enumerate(cleaned_data["instances"][x]):
        p_id = PATH + str(cleaned_data["patient_id"][x]) + "/" + str(cleaned_data["series_id"][x]) + "/" + str(instance) + ".dcm"
        paths.append(p_id)
    total_paths.append(paths)

In [17]:
cleaned_data["instances"] = total_paths

In [18]:
cleaned_data

,patient_id,series_id,instances,category
0,10004,21057,[/kaggle/input/rsna-2023-abdominal-trauma-dete...,1
1,10004,51033,[/kaggle/input/rsna-2023-abdominal-trauma-dete...,1
2,10217,16066,[/kaggle/input/rsna-2023-abdominal-trauma-dete...,1
3,10292,14945,[/kaggle/input/rsna-2023-abdominal-trauma-dete...,1
4,10494,65369,[/kaggle/input/rsna-2023-abdominal-trauma-dete...,1
...,...,...,...,...
254,820,11921,[/kaggle/input/rsna-2023-abdominal-trauma-dete...,1
255,820,38809,[/kaggle/input/rsna-2023-abdominal-trauma-dete...,1
256,8263,30011,[/kaggle/input/rsna-2023-abdominal-trauma-dete...,1
257,9528,1989,[/kaggle/input/rsna-2023-abdominal-trauma-dete...,1


In [19]:
import pydicom
import cv2
from scipy.ndimage import zoom
from sklearn import preprocessing
from glob import glob

def window_converter(image, window_width=400, window_level=50):      
    img_min = window_level - window_width // 2
    img_max = window_level + window_width // 2
    window_image = image.copy()
    window_image[window_image < img_min] = img_min
    window_image[window_image > img_max] = img_max
    #image = (image / image.max() * 255).astype(np.float64)
    return window_image

def transform_to_hu(medical_image, image):
    meta_image = pydicom.dcmread(medical_image)
    intercept = meta_image.RescaleIntercept
    slope = meta_image.RescaleSlope
    hu_image = image * slope + intercept
    return hu_image

def standardize_pixel_array(dcm: pydicom.dataset.FileDataset) -> np.ndarray:
    # Correct DICOM pixel_array if PixelRepresentation == 1.
        pixel_array = dcm.pixel_array
        if dcm.PixelRepresentation == 1:
            bit_shift = dcm.BitsAllocated - dcm.BitsStored
            dtype = pixel_array.dtype 
            pixel_array = (pixel_array << bit_shift).astype(dtype) >> bit_shift
        return pixel_array

def resize_img(img_paths, target_size=(128, 128)):
        volume_shape = (target_size[0], target_size[1], len(img_paths)) 
        volume = np.zeros(volume_shape, dtype=np.float64)
        for i, image_path in enumerate(img_paths):
            image = pydicom.read_file(image_path)
            image = standardize_pixel_array(image)
            hu_image = transform_to_hu(image_path, image)
            window_image = window_converter(hu_image)
            image = cv2.resize(window_image, target_size)
            volume[:,:,i] = image
        return volume
    
def change_depth_siz(patient_volume, target_depth=40):
    desired_depth = target_depth
    current_depth = patient_volume.shape[-1]
    depth = current_depth / desired_depth
    depth_factor = 1 / depth
    img_new = zoom(patient_volume, (1, 1, depth_factor), mode='nearest')
    return img_new
    
def normalize_volume(resized_volume):
    original_shape = resized_volume.shape
    flattened_image = resized_volume.reshape((-1,))
    scaler = preprocessing.MinMaxScaler()
    normalized_flattened_image = scaler.fit_transform(flattened_image.reshape((-1, 1)))
    normalized_volume_image = normalized_flattened_image.reshape(original_shape)
    return normalized_volume_image

def generate_patient_processed_data(list_img_paths, list_labels, target_size=(128,128), target_depth=40):

    num_patients = len(list_img_paths)
    height = target_size[0]
    width = target_size[1]
    depth = target_depth

    volume_array = np.zeros((height, width, depth), dtype=np.float64)
    labels_array = np.array(list_labels, dtype=np.float64)

    print("Initializing data preprocessing with the following dimensions-> Volumes:{} Labels:{}".format(volume_array.shape, labels_array.shape))

    resized_images = resize_img(list_img_paths, target_size=target_size)
    siz_volume = change_depth_siz(resized_images)
    normalized_siz_volume = normalize_volume(siz_volume)

    volume_array = normalized_siz_volume

    return volume_array, labels_array

In [21]:
#volume, label = generate_patient_processed_data(paths, cleaned_data["category"][0], target_size=(128,128), target_depth=40)

Initializing data preprocessing with the following dimensions-> Volumes:(128, 128, 40) Labels:()


In [22]:
#volume.shape

(128, 128, 40)

In [20]:
"""
rows = 10  # Number of rows in the subplot grid
cols = 4  # Number of columns in the subplot grid
depth = 40

fig, axes = plt.subplots(rows, cols, figsize=(8, 44))
#fig.suptitle(f'3D Volume Slices')

for d in range(depth):
    row = d // cols
    col = d % cols
    
    ax = axes[row, col]
    ax.imshow(volume[:,:,d], cmap='gray')  # You can change the colormap if needed
    ax.set_title(f"Slice {d}")
    ax.axis('off')  # Turn off axis for cleaner visualization

# Adjust layout
plt.tight_layout()
plt.subplots_adjust(top=0.5) # Adjust the position of the suptitle

# Show the plot
plt.show()
"""

'\nrows = 10  # Number of rows in the subplot grid\ncols = 4  # Number of columns in the subplot grid\ndepth = 40\n\nfig, axes = plt.subplots(rows, cols, figsize=(8, 44))\n#fig.suptitle(f\'3D Volume Slices\')\n\nfor d in range(depth):\n    row = d // cols\n    col = d % cols\n    \n    ax = axes[row, col]\n    ax.imshow(volume[:,:,d], cmap=\'gray\')  # You can change the colormap if needed\n    ax.set_title(f"Slice {d}")\n    ax.axis(\'off\')  # Turn off axis for cleaner visualization\n\n# Adjust layout\nplt.tight_layout()\nplt.subplots_adjust(top=0.5) # Adjust the position of the suptitle\n\n# Show the plot\nplt.show()\n'

## We have the positive data, let´s get some negative patients for the extravasation injury

In [37]:
data = pd.read_csv("/kaggle/input/rsna-2023-abdominal-trauma-detection/train.csv")
data = data.loc[data["extravasation_injury"] == 0]
data = data.sample(frac=1, random_state=42).reset_index(drop=True)


cat_data = pd.read_csv("/kaggle/input/rsna-2023-abdominal-trauma-detection/train_series_meta.csv")

In [38]:
import re

def extract_number_from_path(path):
    match = re.search(r'(\d+)\.dcm$', path)
    if match:
        return int(match.group(1))
    return 0

def get_data_for_3d_volumes(data, train_data_cat, path, number_idx):

    data_to_merge = data[["patient_id", "series_id"]]
    patient_category = train_data_cat[["patient_id", "extravasation_injury"]]
    
    merged_df = data_to_merge.merge(patient_category, on='patient_id', how='left')
    
    shuffled_data = merged_df.sample(frac=1, random_state=42)
    shuffled_indexes = shuffled_data.index[:number_idx]
    selected_rows = shuffled_data.loc[shuffled_indexes]
    data_to_merge_processed = selected_rows.reset_index()
    
    total_paths = []
    patient_ids = []
    series_ids = []
    category = [0 for i in range(number_idx)]
    
    for patient_id in range(len(data_to_merge_processed)):
    
        p_id = str(data_to_merge_processed["patient_id"][patient_id]) + "/" + str(data_to_merge_processed["series_id"][patient_id])
        str_imgs_path = path + p_id + '/'
        patient_img_paths = []

        for file in glob(str_imgs_path + '/*'):
            patient_img_paths.append(file)
        
        
        sorted_file_paths = sorted(patient_img_paths, key=extract_number_from_path)
        total_paths.append(sorted_file_paths)
        patient_ids.append(data_to_merge_processed["patient_id"][patient_id])
        series_ids.append(data_to_merge_processed["series_id"][patient_id])
        category.append(data_to_merge_processed["extravasation_injury"][patient_id])
    
    final_data = pd.DataFrame(list(zip(patient_ids, series_ids, total_paths, category)),
               columns =["satient_id","series_id", "instances", "category"])
    
    return final_data

In [39]:
path = "/kaggle/input/rsna-2023-abdominal-trauma-detection/train_images/"
data_negative_patients = get_data_for_3d_volumes(cat_data, data, path, number_idx=256)
data_negative_patients 

,satient_id,series_id,instances,category
0,25102,50875,[/kaggle/input/rsna-2023-abdominal-trauma-dete...,0
1,31158,28163,[/kaggle/input/rsna-2023-abdominal-trauma-dete...,0
2,51177,42398,[/kaggle/input/rsna-2023-abdominal-trauma-dete...,0
3,56046,38794,[/kaggle/input/rsna-2023-abdominal-trauma-dete...,0
4,56400,12546,[/kaggle/input/rsna-2023-abdominal-trauma-dete...,0
...,...,...,...,...
251,48915,37971,[/kaggle/input/rsna-2023-abdominal-trauma-dete...,0
252,16691,60898,[/kaggle/input/rsna-2023-abdominal-trauma-dete...,0
253,904,10621,[/kaggle/input/rsna-2023-abdominal-trauma-dete...,0
254,56457,2285,[/kaggle/input/rsna-2023-abdominal-trauma-dete...,0


## Data to be mapped

In [26]:
cleaned_data.head()

,patient_id,series_id,instances,category
0,10004,21057,[/kaggle/input/rsna-2023-abdominal-trauma-dete...,1
1,10004,51033,[/kaggle/input/rsna-2023-abdominal-trauma-dete...,1
2,10217,16066,[/kaggle/input/rsna-2023-abdominal-trauma-dete...,1
3,10292,14945,[/kaggle/input/rsna-2023-abdominal-trauma-dete...,1
4,10494,65369,[/kaggle/input/rsna-2023-abdominal-trauma-dete...,1


In [27]:
data_negative_patients.head()

,satient_id,series_id,instances,category
0,25102,50875,[/kaggle/input/rsna-2023-abdominal-trauma-dete...,0
1,31158,28163,[/kaggle/input/rsna-2023-abdominal-trauma-dete...,0
2,51177,42398,[/kaggle/input/rsna-2023-abdominal-trauma-dete...,0
3,56046,38794,[/kaggle/input/rsna-2023-abdominal-trauma-dete...,0
4,56400,12546,[/kaggle/input/rsna-2023-abdominal-trauma-dete...,0


## Data generation

In [29]:
for i in range(len(cleaned_data)):
    patient_data_volumes, _ = generate_patient_processed_data(cleaned_data["instances"][i], cleaned_data["category"][i], target_size=(128,128), target_depth=40)

    with open(f'/kaggle/working/{str(cleaned_data["patient_id"][i])}_{str(cleaned_data["series_id"][i])}.npy', 'wb') as f:
        np.save(f, patient_data_volumes)

Initializing data preprocessing with the following dimensions-> Volumes:(128, 128, 40) Labels:()
Initializing data preprocessing with the following dimensions-> Volumes:(128, 128, 40) Labels:()
Initializing data preprocessing with the following dimensions-> Volumes:(128, 128, 40) Labels:()
Initializing data preprocessing with the following dimensions-> Volumes:(128, 128, 40) Labels:()
Initializing data preprocessing with the following dimensions-> Volumes:(128, 128, 40) Labels:()
Initializing data preprocessing with the following dimensions-> Volumes:(128, 128, 40) Labels:()
Initializing data preprocessing with the following dimensions-> Volumes:(128, 128, 40) Labels:()
Initializing data preprocessing with the following dimensions-> Volumes:(128, 128, 40) Labels:()
Initializing data preprocessing with the following dimensions-> Volumes:(128, 128, 40) Labels:()
Initializing data preprocessing with the following dimensions-> Volumes:(128, 128, 40) Labels:()
Initializing data preprocessin

In [31]:
for i in range(len(data_negative_patients)):
    patient_data_volumes, _ = generate_patient_processed_data(data_negative_patients["instances"][i], data_negative_patients["category"][i], target_size=(128,128), target_depth=40)

    with open(f'/kaggle/working/{str(data_negative_patients["satient_id"][i])}_{str(data_negative_patients["series_id"][i])}.npy', 'wb') as f:
        np.save(f, patient_data_volumes)

Initializing data preprocessing with the following dimensions-> Volumes:(128, 128, 40) Labels:()
Initializing data preprocessing with the following dimensions-> Volumes:(128, 128, 40) Labels:()
Initializing data preprocessing with the following dimensions-> Volumes:(128, 128, 40) Labels:()
Initializing data preprocessing with the following dimensions-> Volumes:(128, 128, 40) Labels:()
Initializing data preprocessing with the following dimensions-> Volumes:(128, 128, 40) Labels:()
Initializing data preprocessing with the following dimensions-> Volumes:(128, 128, 40) Labels:()
Initializing data preprocessing with the following dimensions-> Volumes:(128, 128, 40) Labels:()
Initializing data preprocessing with the following dimensions-> Volumes:(128, 128, 40) Labels:()
Initializing data preprocessing with the following dimensions-> Volumes:(128, 128, 40) Labels:()
Initializing data preprocessing with the following dimensions-> Volumes:(128, 128, 40) Labels:()
Initializing data preprocessin

## Data generator

In [55]:
import tensorflow as tf 
import pydicom
import cv2
from scipy.ndimage import zoom
from sklearn import preprocessing
import math
import numpy as np


class NumpyImage3DGenerator(tf.keras.utils.Sequence):

    def __init__(self, patient_set, series_set, category_set, batch_size):
        self.x, self.y = patient_set, category_set
        self.series = series_set
        self.batch_size = batch_size
    
    def __len__(self):
        return math.ceil(len(self.x) / self.batch_size)
    
    def __getitem__(self, idx):
        batch_x = self.x[idx * self.batch_size:(idx + 1) * self.batch_size]
        batch_y = self.y[idx * self.batch_size:(idx + 1) * self.batch_size]
        batch_of_volumes = []
        for patient in range(len(batch_x)):
            try:
                with open(f'/kaggle/working/{self.x[patient]}_{self.series[patient]}.npy', 'rb') as f:
                    X = np.load(f, allow_pickle=True)
                batch_of_volumes.append(X)
            except:
                continue
                
        return np.array(batch_of_volumes, dtype=np.float64), np.array(batch_y, dtype=np.float64)

## Model definition and training

In [60]:
import tensorflow as tf 
from keras import backend as K
from keras import backend as K
from keras.optimizers import SGD
from keras.models import load_model, Model
from keras.layers import Conv3D, MaxPool3D, Flatten, Dense
from keras.layers import Dropout, Input, BatchNormalization



class ThreeDCNN:
    def __init__(self, input_shape):
        self.input_shape = input_shape
        self.model = self.build_model()

    def convolutional_block_3d(self, inputs, num_filters):
        x = Conv3D(filters=num_filters, kernel_size=(3, 3, 3), activation="relu")(inputs)
        x = MaxPool3D(pool_size=(2, 2, 2), padding='same')(x)
        x = BatchNormalization()(x)
        return x

    def dense_block(self, flatten_layer):

        initializer = tf.keras.initializers.RandomNormal(mean=0., stddev=1.)

        dense_layer_1 = Dense(units=512, kernel_initializer=initializer, activation='relu')(flatten_layer)
        dense_layer_1 = Dropout(0.4)(dense_layer_1)
        dense_layer_2 = Dense(units=256, activation='relu')(dense_layer_1)
        dense_layer_2 = Dropout(0.4)(dense_layer_2)
        output_layer = Dense(units=2, activation='softmax')(dense_layer_2)
        return output_layer
    
    def compile_model(self, model):
        #optimizer = SGD(learning_rate=1e-06, momentum=0.99, decay=0.0, nesterov=False)
        model.compile(loss="mae", optimizer=tf.keras.optimizers.Adam(), metrics=['acc'])
    
    def summary(self):
        self.model.summary()

    def build_model(self):
        input_layer = Input(self.input_shape)
        x1 = self.convolutional_block_3d(input_layer, 64)
        x2 = self.convolutional_block_3d(x1, 64)
        x3 = self.convolutional_block_3d(x2, 128)
        x4 = self.convolutional_block_3d(x3, 256)
        flatten_layer = Flatten()(x4)
        output = self.dense_block(flatten_layer)
        model = Model(inputs=input_layer, outputs=output)
        self.compile_model(model)
        return model

## Data splitting

In [49]:
data_negative_patients.rename(columns = {'satient_id':'patient_id'}, inplace = True)

data = pd.concat([cleaned_data[["patient_id", "series_id", "category"]], data_negative_patients[["patient_id", "series_id", "category"]]], ignore_index=True)
data = data.sample(frac=1)
data

,patient_id,series_id,category
148,42232,58540,1
351,521,25315,0
17,12958,18631,1
87,28976,31146,1
346,57612,6072,0
...,...,...,...
513,56457,2285,0
400,65438,35647,0
430,16439,16607,0
311,15949,48567,0


In [56]:
def create_pattern(data):
    pattern = [1, 1, 0, 0]
    result = []
    
    while len(result) < len(data):
        result.extend(pattern)
    
    return result[:len(data)]

np.random.seed(10)

rnd = np.random.rand(len(data))
train = data[rnd<0.8]
test = data[(rnd>=0.8)]
    
train = train.reset_index(drop=True)
test = test.reset_index(drop=True)

train['pattern'] = create_pattern(train)
train.sort_values(by=['pattern'], inplace=True)
train.reset_index(drop=True, inplace=True)


data_gen = NumpyImage3DGenerator(train["patient_id"], train["series_id"], train["category"], batch_size=8)
data_gen_test = NumpyImage3DGenerator(test["patient_id"], test["series_id"],test["category"], batch_size=8)

In [51]:
train

,patient_id,series_id,category,pattern
0,16439,16607,0,0
1,29978,39293,1,0
2,47065,39222,1,0
3,10917,30843,0,0
4,19366,3532,1,0
...,...,...,...,...
407,14321,8413,1,1
408,33251,17605,1,1
409,19742,4589,1,1
410,37942,63754,1,1


In [58]:
x, y = data_gen[0]

In [59]:
x.shape

(8, 128, 128, 40)

In [62]:
input_shape = (128, 128, 40, 1)
model = ThreeDCNN(input_shape).model
history = model.fit(data_gen, validation_data=data_gen_test, epochs=100)

Epoch 1/100
52/52 [==============================] - 26s 257ms/step - loss: 0.5000 - acc: 0.5316 - val_loss: 0.5000 - val_acc: 0.4272
Epoch 2/100
52/52 [==============================] - 12s 221ms/step - loss: 0.5000 - acc: 0.5267 - val_loss: 0.5000 - val_acc: 0.4272
Epoch 3/100
52/52 [==============================] - 11s 220ms/step - loss: 0.5000 - acc: 0.4733 - val_loss: 0.5000 - val_acc: 0.4272
Epoch 4/100
52/52 [==============================] - 12s 221ms/step - loss: 0.5000 - acc: 0.5194 - val_loss: 0.5000 - val_acc: 0.4272
Epoch 5/100
52/52 [==============================] - 11s 220ms/step - loss: 0.5000 - acc: 0.5485 - val_loss: 0.5000 - val_acc: 0.4272
Epoch 6/100
52/52 [==============================] - 11s 219ms/step - loss: 0.5000 - acc: 0.5316 - val_loss: 0.5000 - val_acc: 0.4272
Epoch 7/100
52/52 [==============================] - 12s 223ms/step - loss: 0.5000 - acc: 0.5510 - val_loss: 0.5000 - val_acc: 0.4272
Epoch 8/100
52/52 [==============================] - 12s 221ms

KeyboardInterrupt: 